<a href="https://colab.research.google.com/github/Keshav1516/stock_price_prediction_with_lstm/blob/main/Stock_Price_Prediction_with_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
# Important Libraries
import pandas as pd
import numpy as np
import yfinance as yf
import datetime
from datetime import date, timedelta
import plotly.graph_objects as go

In [18]:
# Loading the datset in our environement.
today= date.today()

df1= today.strftime("%Y-%m-%d")
end_date= df1

d2= date.today() - timedelta(days=5000)
d2= d2.strftime("%Y-%m-%d")
start_date= d2

data= yf.download("AAPL", start= start_date, end=end_date, progress=False)
data["Date"]= data.index
cols= data[["Date", "Open", "High", "Low", "Close", "Volume"]]
available_cols = [c for c in cols if c in data.columns]

data = data[available_cols]
data.reset_index(drop=True, inplace=True)
data.head()

/tmp/ipython-input-2172179853.py:10: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data= yf.download("AAPL", start= start_date, end=end_date, progress=False)


Price,Date,Open,High,Low,Close,Volume
Ticker,,AAPL,AAPL,AAPL,AAPL,AAPL
0,2011-12-14,11.608778,11.629192,11.337997,11.413347,406887600
1,2011-12-15,11.507614,11.519923,11.356913,11.375826,256200000
2,2011-12-16,11.418451,11.532227,11.394735,11.438264,421478400
3,2011-12-19,11.481797,11.553245,11.422056,11.473991,235530400
4,2011-12-20,11.640600,11.890969,11.625590,11.886465,337215200


In [34]:
# Checking dataset summary.
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3441 entries, 0 to 3440
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   (Date, )        3441 non-null   datetime64[ns]
 1   (Open, AAPL)    3441 non-null   float64       
 2   (High, AAPL)    3441 non-null   float64       
 3   (Low, AAPL)     3441 non-null   float64       
 4   (Close, AAPL)   3441 non-null   float64       
 5   (Volume, AAPL)  3441 non-null   int64         
dtypes: datetime64[ns](1), float64(4), int64(1)
memory usage: 161.4 KB
None


In [38]:
# Flatten the MultiIndex and keep only the first level ("Price")
data.columns = data.columns.droplevel('Ticker')

# Remove extra spaces in column names (if any)
data.columns = data.columns.str.strip()

print(data.columns)

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Volume'], dtype='object', name='Price')


In [39]:
figure= go.Figure(data=[go.Candlestick(x=data["Date"], open= data["Open"], high= data["High"], low= data["Low"],close= data["Close"])])
figure.update_layout(title="Apple Stock Price Analysis", xaxis_rangeslider_visible=False)
figure.show()

In [40]:
#Checking the Correlation.
correlation= data.corr()
print(correlation["Close"].sort_values(ascending=False))

Price
Close     1.000000
High      0.999883
Low       0.999877
Open      0.999744
Date      0.926458
Volume   -0.528601
Name: Close, dtype: float64


In [45]:
# Training LSTM for Stock Price Prediction
x= data[["Open", "High", "Low", "Volume"]]
y= data["Close"]
x= x.to_numpy()
y= y.to_numpy()

In [46]:
print(y.shape)

(3441,)


In [47]:
y= y.reshape(-1,1)

In [48]:
print(y.shape)

(3441, 1)


In [49]:
from sklearn.model_selection import train_test_split

xtrain, xtest, ytrain, ytest= train_test_split(x, y, test_size=0.2, random_state=42)

In [55]:
# Building Neural Network Architecture Model for LSTM

from keras.models import Sequential
from keras.layers import Dense, LSTM
model= Sequential()
model.add(LSTM(128, return_sequences=True, input_shape=(xtrain.shape[1],1)))
model.add(LSTM(64, return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))
model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning:

Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.



Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_2 (LSTM)                   │ (None, 4, 128)         │        66,560 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 25)             │         1,625 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            26 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 117,619 (459.45 KB)

 Trainable params: 117,619 (459.45 KB)

 Non-trainable params: 0 (0.00 B)

In [57]:
# Compiling the Model.
model.compile(optimizer="adam", loss="mean_squared_error")
model.fit(xtrain, ytrain, batch_size=1, epochs=40)

Epoch 1/40
2752/2752 ━━━━━━━━━━━━━━━━━━━━ 45s 7ms/step - loss: 9.1036
Epoch 2/40
2752/2752 ━━━━━━━━━━━━━━━━━━━━ 21s 7ms/step - loss: 12.6780
Epoch 3/40
2752/2752 ━━━━━━━━━━━━━━━━━━━━ 21s 8ms/step - loss: 7.9591
Epoch 4/40
2752/2752 ━━━━━━━━━━━━━━━━━━━━ 18s 7ms/step - loss: 7.6216
Epoch 5/40
2752/2752 ━━━━━━━━━━━━━━━━━━━━ 20s 7ms/step - loss: 10.4008
Epoch 6/40
2752/2752 ━━━━━━━━━━━━━━━━━━━━ 19s 7ms/step - loss: 8.2020
Epoch 7/40
2752/2752 ━━━━━━━━━━━━━━━━━━━━ 19s 7ms/step - loss: 10.1776
Epoch 8/40
2752/2752 ━━━━━━━━━━━━━━━━━━━━ 18s 7ms/step - loss: 6.7113
Epoch 9/40
2752/2752 ━━━━━━━━━━━━━━━━━━━━ 21s 7ms/step - loss: 7.6606
Epoch 10/40
2752/2752 ━━━━━━━━━━━━━━━━━━━━ 19s 7ms/step - loss: 7.0436
Epoch 11/40
2752/2752 ━━━━━━━━━━━━━━━━━━━━ 31s 11ms/step - loss: 7.9217
Epoch 12/40
2752/2752 ━━━━━━━━━━━━━━━━━━━━ 44s 12ms/step - loss: 6.8686
Epoch 13/40
2752/2752 ━━━━━━━━━━━━━━━━━━━━ 26s 9ms/step - loss: 8.0106
Epoch 14/40
2752/2752 ━━━━━━━━━━━━━━━━━━━━ 18s 7ms/step - loss: 7.4476
Epoch 15/4

In [61]:
# features= [Open, High, Low, Volumne]
features=np.array([[226.270004,	226.520004,	223.779999,	30593700]])
model.predict(features)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step


array([[223.23898]], dtype=float32)